#### DiC Assignment 2

Group 6
Members:
 Theresa Mayer
 Theresa Bruckner
 Jan Tölken
 Can Kenan Kandil 
 Thomas Klar


## Part 1

## Part 2

In [48]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [50]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover, IDF, ChiSqSelector, IndexToString, StringIndexer, CountVectorizer, Normalizer
from pyspark.sql import SparkSession

In [52]:
spark = SparkSession.builder.appName("Assignment2").config("spark.executor.memory", "8g").config("spark.driver.memory", "8g").config("spark.python.worker.memory", "4g").config("spark.sql.shuffle.partitions", "10").getOrCreate()

In [55]:
path = "reviews_devset.json" #"hdfs:///user/dic25_shared/amazon-reviews/full/reviews_devset.json"
input_file = spark.read.format("json").load(path).select("category", "reviewText")

In [56]:
input_file.show(n=5)

+--------------------+--------------------+
|            category|          reviewText|
+--------------------+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...|
|Patio_Lawn_and_Garde|This is a very ni...|
|Patio_Lawn_and_Garde|The metal base wi...|
|Patio_Lawn_and_Garde|For the most part...|
|Patio_Lawn_and_Garde|This hose is supp...|
+--------------------+--------------------+
only showing top 5 rows



### Label Encoding

In [58]:
indexer = StringIndexer(inputCol="category", outputCol="label")
indexModel = indexer.fit(input_file)
input_file_1 = indexModel.transform(input_file)

In [60]:
reindexer = IndexToString(inputCol=indexer.getOutputCol(), outputCol="category_reindexed")
reindexer.transform(input_file_1).show(n=5)

+--------------------+--------------------+-----+--------------------+
|            category|          reviewText|label|  category_reindexed|
+--------------------+--------------------+-----+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|For the most part...| 18.0|Patio_Lawn_and_Garde|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|Patio_Lawn_and_Garde|
+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



### Tokenization

In [63]:
tokenizer = RegexTokenizer(inputCol='reviewText', outputCol='tokens', pattern=r"[ \t\d(){}\[\].!?;:,\-=\"~#@&*%€$§\\'\n\r\/]+", minTokenLength=2, toLowercase=True)

In [64]:
input_2 = tokenizer.transform(input_file_1)
input_2.show(n=5)

+--------------------+--------------------+-----+--------------------+
|            category|          reviewText|label|              tokens|
+--------------------+--------------------+-----+--------------------+
|Patio_Lawn_and_Garde|This was a gift f...| 18.0|[this, was, gift,...|
|Patio_Lawn_and_Garde|This is a very ni...| 18.0|[this, is, very, ...|
|Patio_Lawn_and_Garde|The metal base wi...| 18.0|[the, metal, base...|
|Patio_Lawn_and_Garde|For the most part...| 18.0|[for, the, most, ...|
|Patio_Lawn_and_Garde|This hose is supp...| 18.0|[this, hose, is, ...|
+--------------------+--------------------+-----+--------------------+
only showing top 5 rows



### Stopword Removal

In [67]:
stopword_file = "stopwords.txt"
with open(stopword_file, 'r', encoding='utf-8') as f:
    # Strip whitespace and convert to lowercase
    stopwords = [line.strip() for line in f]

In [69]:
stopword_remover = StopWordsRemover(inputCol=tokenizer.getOutputCol(), 
                                    outputCol="tokens_nostop",
                                    stopWords=stopwords)

In [71]:
input_3 = stopword_remover.transform(input_2)
input_3.select("tokens", "tokens_nostop").show(n=5)

+--------------------+--------------------+
|              tokens|       tokens_nostop|
+--------------------+--------------------+
|[this, was, gift,...|[gift, husband, m...|
|[this, is, very, ...|[nice, spreader, ...|
|[the, metal, base...|[metal, base, hos...|
|[for, the, most, ...|[part, works, pre...|
|[this, hose, is, ...|[hose, supposed, ...|
+--------------------+--------------------+
only showing top 5 rows



### TF-IDF Calculation

In [76]:
tf = CountVectorizer(inputCol=stopword_remover.getOutputCol(), 
                      outputCol="tf_output", 
                      vocabSize=40_000)

In [77]:
idf = IDF(inputCol=tf.getOutputCol(), 
          outputCol="tfidf_output",
          minDocFreq=4)

In [79]:
tfmodel = tf.fit(input_3)
input_4 = tfmodel.transform(input_3)
input_4.select("tokens_nostop", "tf_output").show(n=5)

+--------------------+--------------------+
|       tokens_nostop|           tf_output|
+--------------------+--------------------+
|[gift, husband, m...|(40000,[2,3,7,8,3...|
|[nice, spreader, ...|(40000,[0,1,3,21,...|
|[metal, base, hos...|(40000,[4,10,29,1...|
|[part, works, pre...|(40000,[1,3,4,9,1...|
|[hose, supposed, ...|(40000,[12,32,42,...|
+--------------------+--------------------+
only showing top 5 rows



In [82]:
idfModel = idf.fit(input_4)
input_5 = idfModel.transform(input_4)
input_5.select("tf_output", "tfidf_output").show(n=5)

+--------------------+--------------------+
|           tf_output|        tfidf_output|
+--------------------+--------------------+
|(40000,[2,3,7,8,3...|(40000,[2,3,7,8,3...|
|(40000,[0,1,3,21,...|(40000,[0,1,3,21,...|
|(40000,[4,10,29,1...|(40000,[4,10,29,1...|
|(40000,[1,3,4,9,1...|(40000,[1,3,4,9,1...|
|(40000,[12,32,42,...|(40000,[12,32,42,...|
+--------------------+--------------------+
only showing top 5 rows



### Selection of top 2000 features

In [84]:
chisq = ChiSqSelector(featuresCol=idf.getOutputCol(),
                      labelCol="label",
                      outputCol="features",
                      numTopFeatures=2000)

In [85]:
chisqModel = chisq.fit(input_5)
input_6 = chisqModel.transform(input_5)
input_6.select("features").show(n=5)

+--------------------+
|            features|
+--------------------+
|(2000,[2,3,7,8,35...|
|(2000,[0,1,3,21,3...|
|(2000,[4,10,174,3...|
|(2000,[1,3,4,9,10...|
|(2000,[12,29,101,...|
+--------------------+
only showing top 5 rows



### Pipeline Creation

In [87]:
def get_pipeline(n_features=2000):
    chisq.setNumTopFeatures(n_features)
    pipeline = Pipeline(stages=[
        indexer,
        tokenizer,
        stopword_remover,
        tf,
        idf,
        chisq
    ])
    return pipeline

In [88]:
pipeline = get_pipeline(n_features=2000)
preprocessing_pipeline = pipeline.fit(input_file)
preprocessing_pipeline.transform(input_file).select("label", "features").show(n=5)

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 18.0|(2000,[2,3,7,8,35...|
| 18.0|(2000,[0,1,3,21,3...|
| 18.0|(2000,[4,10,174,3...|
| 18.0|(2000,[1,3,4,9,10...|
| 18.0|(2000,[12,29,101,...|
+-----+--------------------+
only showing top 5 rows



### Export most important tokens to file

In [90]:
def get_top_terms_from_pipeline(pipeline):
    n = len(pipeline.stages[5].selectedFeatures)


    vocab = pipeline.stages[3].vocabulary.copy()
    top_words = " ".join(sorted([vocab[i] for i in pipeline.stages[5].selectedFeatures]))
    
    with open("output_ds.txt", "w") as f:
        f.write(top_words)
        
    return n

In [91]:
get_top_terms_from_pipeline(preprocessing_pipeline)

2000

# Part 3

Import the necessary libraries from pyspark

In [94]:
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [95]:
train_data, val_data, test_data = input_file.randomSplit([0.6,0.2,0.2], seed = 1234)

In [96]:
normalizer_2000 = Normalizer(inputCol="features", outputCol="norm_features_2000", p=2.0)
normalizer_500 = Normalizer(inputCol="features_chi_500", outputCol="norm_features_500", p=2.0)

In [97]:
svm = LinearSVC(maxIter = 10, regParam = 0.1, featuresCol = "norm_features_2000")
ovr = OneVsRest(classifier = svm, labelCol = "label",featuresCol = "norm_features_2000")

In [98]:
pipeline = Pipeline(stages=[
        indexer,
        tokenizer,
        stopword_remover,
        tf,
        idf,
        chisq, 
        normalizer_2000,
        ovr
    ])

### Evaluation 

In [99]:
# ParamGrid und Evaluator
paramGrid = (ParamGridBuilder()
    .addGrid(svm.regParam, [0.01, 0.1, 1.0])
    .addGrid(svm.maxIter, [10, 100])
    .addGrid(svm.standardization, [True, False])
    .addGrid(ovr.featuresCol, ["norm_features_2000", "norm_features_500"])
    .build())

In [100]:
evaluator = MulticlassClassificationEvaluator(labelCol = "label", predictionCol = "prediction",metricName="f1")

In [101]:
cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=3,
                    seed=1234)

In [102]:
cvModel = cv.fit(train_data)

Py4JJavaError: An error occurred while calling o26299.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 14509.0 failed 1 times, most recent failure: Lost task 3.0 in stage 14509.0 (TID 72529) (host.docker.internal executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD

In [ ]:
val_predictions = cvModel.transform(val_data)
val_f1 = evaluator.evaluate(val_predictions)
print(f"F1-Score: {val_f1}")